In [ ]:
import os
import math
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import airfoil as ar

%matplotlib inline

The purpose of this notebook is to model the motion of a flapping fish in python but in a way that can be directly translated to an Ansys Fluent UDF function

First we use the airfoil functions to make a NACA0012 airfoil

A fish body can be modeled using the function **naca4(x, t, c, m=0.0, p=0.30, a4=True)** where, 
    **x** is a 1D numpy array of x-coordinates, 
    **c** is the chord length,
    **t** is the maximum thickness,
    **m** is the maximum camber (defaul is symmetric),
    **p** is the location of maximum camber (should be a non-zero number), and 
    **a4** determines if it has an open or closed edge.

In [ ]:
chord = 1.0
# half-cosine spacing
beta = np.linspace(0, np.pi, 100)
#dx = (0.5*(1-np.cos(beta)))
# uniform spacing
dx = np.linspace(0.0, chord, 100)
X, Y = ar.naca4(dx, 0.12, 0.0, 0.30, True, chord)
x, y = ar.convert_to_coordinates(X, Y)

In [ ]:
plt.rcParams.update({'font.size':20})

# plot geometry
width = 10
plt.figure(figsize=(width, width))
plt.grid()
plt.xlabel('x', fontsize=16)
plt.ylabel('y', fontsize=16)
plt.plot(x, y, color='k', linestyle='-', linewidth=2)
plt.axis('scaled')
plt.xlim(-0.1, 1.1)
plt.ylim(-0.1, 0.1)

Output airfoil to a file for geometry import

Motion as determined by Anderson.

- Oscillation about the one-third chord
- alpha(t) = -arctan(pi St cos(wt)) + theta(t)

In [ ]:
def airfoil_motion(U, St, h0, alpha_0, phi=90, number_cycles=1.5):
    # define frequency from strouhal number
    freq = St*U/(2*h0)
    time = np.linspace(0, number_cycles * (1/freq), 1500)
    omega = 2*np.pi*freq
    # alpha as defined in Anderson et al. 1998
    theta_0 = np.arctan(omega*h0/U) - alpha_0
    alpha = np.arctan(omega*h0*np.cos(omega*time)) - theta_0*np.sin(omega*time + np.deg2rad(phi))
    h_t = h0*np.sin(omega*time)
    theta_t = theta_0*np.sin(omega*time + phi)
    
    #zero_crossings = np.where(np.diff(np.sign(alpha)))[0]
    #alpha_cycle = alpha[zero_crossings[0]+1:zero_crossings[-1]+1]
    #time_cycle = time[zero_crossings[0]+1:zero_crossings[-1]+1]
    #time_cycle = time_cycle - time_cycle[0]

    #return time, alpha, np.rad2deg(alpha)
    return time, h_t, theta_t, alpha

In [ ]:
# define the freestream from Reynolds number
Re = 40000
chord = 1.0

U = 1.0

Re = 40000
visc = U * chord / Re

phi = np.deg2rad(90)

h0 = 0.75 * chord

alpha_0 = np.deg2rad(15)

Strouhal = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]

fig1, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
for St in Strouhal:
    time, h_t, theta_t, alpha_t = airfoil_motion(U, St, h0, alpha_0, number_cycles=2.5)
    ax1.plot(time, h_t, label='St = ' + str(St))
    ax2.plot(time, np.rad2deg(alpha_t), label='St= ' + str(St))

ax1.set_xlabel('time [s]')
ax1.set_ylabel('h(t) [m]')

ax2.set_xlabel('time [s]')
ax2.set_ylabel(r'$ \alpha (t)\; [^o] $')
ax1.legend(ncol=3, bbox_to_anchor=(1.0, 1.15))
ax2.legend(ncol=3, bbox_to_anchor=(1.0, 1.15))

In [ ]:
St = 0.2
chord = 1.0
U = 1.0
h0 = 0.75 * chord

alpha_0s = [5, 10, 15, 20, 25, 30]
fig3, (ax3, ax4) = plt.subplots(1, 2, figsize=(20,10))

for alpha_0 in alpha_0s:
    time, h_t, theta_t, alpha_t = airfoil_motion(U, St, h0, np.deg2rad(alpha_0), number_cycles=2.5)

    ax3.plot(time, h_t, label=r'$ \alpha_0 $= ' + str(np.round(alpha_0,3)))
    ax4.plot(time, np.rad2deg(alpha_t), label=r'$ \alpha_0 $= ' + str(np.round(alpha_0,3)))

ax3.set_ylabel('h(t) [m]')
ax3.set_xlabel('time [s]')

ax4.set_ylabel(r'$ \alpha (t) \; [^o]$')
ax4.set_xlabel('time [s]')

ax3.legend(ncol=3, bbox_to_anchor=(1.0, 1.15))
ax4.legend(ncol=3, bbox_to_anchor=(1.0, 1.15))

In [ ]:
St = 0.60
chord = 1.0
U = 1.0
h0 = 0.75 * chord

alpha_0 = np.deg2rad(15)
time, h_t, theta_t, alpha_t = airfoil_motion(U, St, h0, alpha_0, number_cycles=2.6)
number_points = time.shape[0]

txt_template = '({times} (({surge} {sway} {heave}) ({roll} {pitch} {yaw}))) \n'
header = '\n' + str(number_points) + '\n' + '( \n'
footer = ')'

with open('6DoF.dat', 'w') as f:
    f.write(header)
    for i, timestep in enumerate(time):
        f.write(txt_template.format(times=np.round(timestep,4), surge=0.0, sway=h_t[i], heave=0.0,
                                    roll=0.0, pitch=0.0, yaw=np.round(np.rad2deg(alpha_t[i]),4)))
    f.write(footer)

In [ ]:
print(time[-1])